# Argovis explore vertical structure of ocean fields

This notebook contains some examples and is a work in progress: code will transition from here to other notebooks. Public consumers should focus on our educational notebooks, starting with [Intro to Argovis](https://github.com/argovis/demo_notebooks/blob/main/Intro_to_Argovis.ipynb).

In [ ]:
from argovisHelpers import helpers as avh
import datetime #, pandas, matplotlib, scipy, numpy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
#from matplotlib import cm
#import cartopy.crs as ccrs

# # for the function
# import xarray as xr
# import dateutil
from Argovis_tasks_helpers import interpolate_profiles
# #import pandas
# # 
from Argovis_tasks_helpers import get_route,show_variable_names_for_collections, format_api_output,get_api_output_formatted_list_1var, api_output_formatted_list_1var_plot_lons_lats_map,api_output_formatted_list_1var_plot_profiles,api_output_formatted_list_1var_plot_map,api_output_formatted_list_1var_plot_horizontal_and_time_ave

API_KEY=''


In [ ]:
# API call showing collections available for each route

#### parameters the user needs to select
# user selects list of collections to use for the plot

#### for bgc variables we have gridded time mean fields
selection_params = {}
selection_params['collections']  = ['argo', 'grids/glodap',
              ] # we compare argo profiles with a gridded product

# selection_params['collections']  = ['cchdo', 'grids/glodap',
#               ] # we compare argo profiles with a gridded product

# selection_params['collections']  = ['argo', 'grids/rg09',
#               ] # we compare argo profiles with a gridded product

show_variable_names_for_collections(collections_list=selection_params['collections'],API_KEY=API_KEY)

#### params varying with collection
# define name of variable of interest for each collection (first show list of all variables available for each collection)
selection_params['varnames']     = ['doxy', 'oxygen']
selection_params['varnames_qc']  = [',1', ''] # argoqc = 1 is best quality
selection_params['varname_title']     = 'Oxygen'

# selection_params['varnames']     = ['doxy', 'oxygen']
# selection_params['varnames_qc']  = [',2', ''] # woceqc = 2 is best quality, it corresponds to 'no problem noted' (see https://dmoserv3.whoi.edu/data_docs/GEOTRACES/EPZT/WOCE_QualityFlags.pdf)
# selection_params['varname_title']     = 'Oxygen, umol/kg'

# selection_params['varnames']     = ['salinity', 'rg09_salinity']
# selection_params['varnames_qc']  = [',1', ''] # argoqc = 1 is best quality
# selection_params['varname_title']     = 'Salinity, psu'

selection_params['data_extra']     = ['',''] # if we wanted to only consider oxygen profiles where temperature is qc 1, e.g. 'temperature,1'
                                           # please note that only the variable in varnames is stored now

# define name of the variable that includes levels for each collection
selection_params['varname_levels'] = ['pressure',''] # for the gridded product, the level info is in the metadata, i.e. there is no variable in 'data' (for argo, 'pressure' is within the 'data' instead)
####

#### params varying with the region
# in this example, we will use the 'box' selection (in the next cell),
# hence we indicate here the bottom/left and top/right vertices... 
# the other option is to search in a 'polygon' and indicate the polygon
# vertices in a list (first and last vertex should be the same)
selection_params['regions']     = [
                        #[[-55.5,35.5],[-45.5,40.5]],
                        [[-145.5,45.5],[-135.5,50.5]],
                        [[-50,45],[-40,50]],
                        ]
selection_params['regions_type'] = ['box', 'box']

selection_params['regions_tag'] = ['Pacific', 'Atlantic']
####

# list of startDate and endDates of interest (note: these will not be used for glodap as glodap only provides a time mean)
selection_params['startDate']    = ['2021-01-01T00:00:00Z']
selection_params['endDate']      = ['2021-12-31T00:00:00Z']

#### other params for the api query
# platform?


#### other params
# levels for vertically integrated profiles
selection_params['interp_levels']= list(range(10,2001))[0::20]
colors = list(mcolors.TABLEAU_COLORS.keys())

In [ ]:
api_output_formatted_list = get_api_output_formatted_list_1var(selection_params=selection_params,API_KEY=API_KEY)

In [ ]:
# plot map of lons and lats for data that are not gridded
api_output_formatted_list_1var_plot_lons_lats_map(api_output_formatted_list)      

In [ ]:
# plot the profiles shown above
api_output_formatted_list_1var_plot_profiles(api_output_formatted_list)

In [ ]:
# plot map at one level and time
api_output_formatted_list_1var_plot_map(api_output_formatted_list,ilev=0,itime=0)  
               

In [ ]:
# plot horizontal and time average
api_output_formatted_list_1var_plot_horizontal_and_time_ave(api_output_formatted_list,colors)  


## The same plots can be created for a specific Argo platform and/or a woce line

In [ ]:
selection_params = {}
selection_params['collections']  = ['argo']
# selection_params['collections']  = ['easyocean']

selection_params['parameter_name']= 'platform'
# selection_params['parameter_name']= 'woceline'

#### object of interest (should be consistent with selection_params['collections'])
selection_params['parameter']     = ['4903274']
# selection_params['parameter']     = ['A10']


# list all the values for the parameter of interest
vocab_params = {
    "parameter": selection_params['parameter_name'] 
}
for icollection in selection_params['collections']:
    print(avh.query(icollection+'/vocabulary', options=vocab_params, apikey=API_KEY, apiroot=get_route(icollection)))

#### params varying with collection
# define name of variable of interest for each collection (first show list of all variables available for each collection)
selection_params['varnames']     = ['doxy']
selection_params['varnames_qc']  = [',1'] # argoqc = 1 is best quality

# test with easyocean
# selection_params['varnames']     = ['doxy']
# selection_params['varnames_qc']  = [''] # woceqc = 2 is best quality

selection_params['varname_title']     = 'Oxygen, umol/kg'
# if we want also other variables:
selection_params['data_extra']   = [',temperature,1,salinity,1']

# define name of the variable that includes levels for each collection
selection_params['varname_levels'] = ['pressure'] # 
####

#### other params
# levels for vertically integrated profiles
selection_params['interp_levels']= list(range(10,2001))[0::20]
selection_params['region_tag']   = ''

# parameters for the query
i = 0

icollection = selection_params['collections'][i]
data_str = selection_params['varnames'][i]+selection_params['varnames_qc'][i]+selection_params['data_extra'][i]

if icollection == 'easyocean':
    data_str = selection_params['varnames'][i]

iparam = {'data': data_str}

iparam[selection_params['parameter_name']] = selection_params['parameter'][i]

if selection_params['parameter_name'] == 'woceline':
    meta_params = {
        "woceline": selection_params['parameter']  # or anything from the list above
    }

    print(avh.query(icollection+'/meta', options=meta_params, apikey=API_KEY, apiroot=get_route(icollection)))

    # how the user assigns the start date should be improved
    iparam['section_start_date'] = '2011-09-28T00:00:00.000Z'


In [ ]:
# let's query data from the selected object
api_output = avh.query(icollection, options=iparam, verbose='true',apikey=API_KEY, apiroot=get_route(icollection))


In [ ]:
# let's create api_output_formatted for each variable that was queried
api_output_formatted_all = {}
api_output_formatted_all_var = []
for ivar in data_str.split(','):
    if not ivar.isnumeric():
        api_output_formatted_all_var.append(ivar)
        api_output_formatted_all[ivar] = format_api_output(api_output,selection_params,ivar,index_collection=0,API_KEY='')
        api_output_formatted_all[ivar]['region_tag'] = ''
        api_output_formatted_all[ivar]['startDate']  = min(api_output_formatted_all[ivar]['timestamp']).strftime("%Y-%m-%d")
        api_output_formatted_all[ivar]['endDate']    = max(api_output_formatted_all[ivar]['timestamp']).strftime("%Y-%m-%d")
        api_output_formatted_all[ivar]['varname_title'] = ivar[0].upper()+ivar[1::]
        

In [ ]:
# let's plot a map for the data locations
api_output_formatted_list_1var_plot_lons_lats_map([api_output_formatted_all[api_output_formatted_all_var[0]]])  

In [ ]:
for ivar in api_output_formatted_all_var:
    # plot the profiles shown above
    api_output_formatted_list_1var_plot_profiles([api_output_formatted_all[ivar]])
    # plot horizontal and time average
    api_output_formatted_list_1var_plot_horizontal_and_time_ave([api_output_formatted_all[ivar]],colors)  


In [ ]:
# line plots for all the points in the xar (using values and flatten?); draw the shape of interest in the maps showing profiles?

In [ ]:
inprogress

In [ ]:
#https://sites.google.com/view/paztronomer/blog/basic/python-colors
month_groups_cols= ['dodgerblue', 'violet', 'orangered', 'gold']
month_groups     = [[12, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]]
month_groups_tags= ['DJF', 'MAM', 'JJA', 'SON']
profiles_in_regions_and_horiz_ave_plot1d_all_col_by_monthgroup(data_reg=data_reg,month_groups=month_groups,month_groups_cols=month_groups_cols,month_groups_tags=month_groups_tags,xlabel_tag=vartitle)


In [ ]:
# bin in time? platform history? woceline? easyocean?